# RNA 3D Folding - Initial Data Exploration

**Competition**: Stanford RNA 3D Folding Part 2  
**Objective**: Predict 3D structure of RNA molecules from sequences  
**Metric**: TM-Score (0 to 1, higher is better)  
**Deadline**: March 25, 2026

## Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from Bio import SeqIO
from Bio.Seq import Seq

# PyTorch
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# Configure plotting
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Paths
DATA_DIR = Path("../data/raw")
PROCESSED_DIR = Path("../data/processed")
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

## 1. Data Overview

In [ ]:
# List data directory contents
if DATA_DIR.exists():
    print("Data directory contents:")
    for item in sorted(DATA_DIR.iterdir()):
        if item.is_dir():
            num_files = len(list(item.iterdir()))
            print(f"  📁 {item.name:30s} ({num_files} files)")
        else:
            size_mb = item.stat().st_size / (1024**2)
            print(f"  📄 {item.name:30s} ({size_mb:.2f} MB)")
else:
    print("⚠️ Data directory not found. Run data download first.")

## 2. MSA File Exploration

Multiple Sequence Alignments are crucial for template-based modeling.

In [ ]:
# Explore MSA files
msa_dir = DATA_DIR / "MSA"

if msa_dir.exists():
    msa_files = list(msa_dir.glob("*.fasta"))
    print(f"Total MSA files: {len(msa_files)}")
    
    # Sample a few files
    print("\nSample MSA files:")
    for msa_file in msa_files[:5]:
        # Read FASTA file
        sequences = list(SeqIO.parse(msa_file, "fasta"))
        
        if sequences:
            seq_lengths = [len(seq.seq) for seq in sequences]
            print(f"\n{msa_file.name}:")
            print(f"  Sequences: {len(sequences)}")
            print(f"  Avg length: {np.mean(seq_lengths):.0f} nucleotides")
            print(f"  Length range: {min(seq_lengths)} - {max(seq_lengths)}")
            print(f"  First sequence: {str(sequences[0].seq)[:60]}...")
else:
    print("⚠️ MSA directory not found")

## 3. Nucleotide Distribution Analysis

In [ ]:
# Analyze nucleotide composition from MSA files
if msa_dir.exists() and msa_files:
    nucleotide_counts = {'A': 0, 'U': 0, 'G': 0, 'C': 0, 'Other': 0}
    total_length = 0
    
    # Sample 100 files for efficiency
    sample_files = msa_files[:100]
    
    for msa_file in sample_files:
        sequences = list(SeqIO.parse(msa_file, "fasta"))
        for seq in sequences:
            seq_str = str(seq.seq).upper()
            total_length += len(seq_str)
            
            for nuc in seq_str:
                if nuc in nucleotide_counts:
                    nucleotide_counts[nuc] += 1
                else:
                    nucleotide_counts['Other'] += 1
    
    # Plot distribution
    fig, ax = plt.subplots(figsize=(10, 6))
    
    nucleotides = list(nucleotide_counts.keys())
    counts = list(nucleotide_counts.values())
    percentages = [count / total_length * 100 for count in counts]
    
    colors = ['#e74c3c', '#3498db', '#2ecc71', '#f39c12', '#95a5a6']
    bars = ax.bar(nucleotides, percentages, color=colors)
    
    ax.set_xlabel('Nucleotide', fontsize=12)
    ax.set_ylabel('Percentage (%)', fontsize=12)
    ax.set_title('Nucleotide Distribution in MSA Files (Sample)', fontsize=14)
    ax.grid(axis='y', alpha=0.3)
    
    # Add percentage labels on bars
    for bar, pct in zip(bars, percentages):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{pct:.1f}%',
                ha='center', va='bottom', fontsize=10)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nAnalyzed {len(sample_files)} MSA files")
    print(f"Total nucleotides: {total_length:,}")

## 4. Sequence Length Distribution

In [ ]:
# Analyze sequence length distribution
if msa_dir.exists() and msa_files:
    all_lengths = []
    
    # Sample files for efficiency
    for msa_file in msa_files[:200]:
        sequences = list(SeqIO.parse(msa_file, "fasta"))
        all_lengths.extend([len(seq.seq) for seq in sequences])
    
    # Plot histogram
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Histogram
    ax1.hist(all_lengths, bins=50, color='#3498db', alpha=0.7, edgecolor='black')
    ax1.set_xlabel('Sequence Length (nucleotides)', fontsize=12)
    ax1.set_ylabel('Frequency', fontsize=12)
    ax1.set_title('Sequence Length Distribution', fontsize=14)
    ax1.grid(axis='y', alpha=0.3)
    
    # Box plot
    ax2.boxplot(all_lengths, vert=True)
    ax2.set_ylabel('Sequence Length (nucleotides)', fontsize=12)
    ax2.set_title('Sequence Length Box Plot', fontsize=14)
    ax2.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nSequence Length Statistics:")
    print(f"  Total sequences: {len(all_lengths):,}")
    print(f"  Mean: {np.mean(all_lengths):.1f} nucleotides")
    print(f"  Median: {np.median(all_lengths):.1f} nucleotides")
    print(f"  Std dev: {np.std(all_lengths):.1f}")
    print(f"  Min: {min(all_lengths)} nucleotides")
    print(f"  Max: {max(all_lengths)} nucleotides")
    print(f"  25th percentile: {np.percentile(all_lengths, 25):.1f}")
    print(f"  75th percentile: {np.percentile(all_lengths, 75):.1f}")

## 5. Next Steps

Based on initial exploration:

1. **Data Loading**: Create efficient data loaders for training
2. **Template Discovery**: Build MSA search pipeline
3. **Feature Engineering**: 
   - One-hot encoding of sequences
   - Positional encoding
   - MSA features (covariation patterns)
4. **Baseline Model**: Simple LSTM/Transformer for coordinate prediction
5. **TM-Score Calculation**: Implement evaluation metric
6. **Template-Based Approach**: Main winning strategy from Part 1

### Key Insights
- Part 1 winners used **template-based modeling** without deep learning
- AlphaFold 3 was beaten by template methods
- MSA files are crucial for template discovery
- Focus on TM-Score optimization

## Save Analysis Results

In [ ]:
# Save exploration summary
if msa_dir.exists() and msa_files:
    summary = {
        'total_msa_files': len(msa_files),
        'nucleotide_distribution': nucleotide_counts,
        'sequence_stats': {
            'count': len(all_lengths),
            'mean_length': float(np.mean(all_lengths)),
            'median_length': float(np.median(all_lengths)),
            'min_length': int(min(all_lengths)),
            'max_length': int(max(all_lengths)),
        }
    }
    
    import json
    with open(PROCESSED_DIR / 'exploration_summary.json', 'w') as f:
        json.dump(summary, f, indent=2)
    
    print("✓ Summary saved to data/processed/exploration_summary.json")